## Creacion de DF

Importamos todas las librerias que vamos a utilizar

In [4]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import StructField, StructType, StringType, IntegerType, StringType, FloatType, Row

El SQLContext nos permite acceder a las diferentes funciones del sql context y poder diferenciarlo en caso de usar diferentes versiones de Spark.

In [5]:
spark = SparkContext(master="local", appName="DataFrames")
sql_context = SQLContext(spark)

21/08/09 11:40:53 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.100.109 instead (on interface enp0s31f6)
21/08/09 11:40:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/08/09 11:40:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Mostramos la cabecera del archivo juegos.csv

In [13]:
!head -n 4 files/juegos.csv

,nombre_juego,annio,temporada,ciudad
1,1896 Verano,1896,Verano,Athina
2,1900 Verano,1900,Verano,Paris
3,1904 Verano,1904,Verano,St. Louis


## Usando un Schema

Creamos el esquema para nuestro DF.

In [6]:
juego_schema = StructType([
  StructField('juego_id', IntegerType(), False),
  StructField('anio', StringType(), False),
  StructField('temporada', StringType(), False),
  StructField('ciudad', StringType(), False),
])

juego_df = sql_context.read.schema(juego_schema) \
    .option("header", True) \
    .csv("files/juegos.csv")

El DF no se crea hasta que ejecutemos una operacion.

In [17]:
juego_df.show(4)

+--------+-----------+---------+------+
|juego_id|       anio|temporada|ciudad|
+--------+-----------+---------+------+
|       1|1896 Verano|     1896|Verano|
|       2|1900 Verano|     1900|Verano|
|       3|1904 Verano|     1904|Verano|
|       4|1906 Verano|     1906|Verano|
+--------+-----------+---------+------+
only showing top 4 rows



21/08/09 11:19:36 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 5, schema size: 4
CSV file: file:///home/will/Projects/spark/files/juegos.csv


In [18]:
spark

<SparkContext master=local appName=DataFrames>

## Usando un RDD
Hay que tomar en cuenta:
- Eliminar el encabezado
- Castear valores a sus respectivos tipos de datos
- Crear el esquema a usar

### Funcion para eliminar el header

In [7]:
def dropFirstRow(index,iterator):
     return iter(list(iterator)[1:])

Primero cargamos los rdds de deportista

In [18]:
deportista = spark.textFile("files/deportista.csv").map(lambda l: l.split(','))
deportista2 = spark.textFile("files/deportista2.csv").map(lambda l: l.split(','))
deportista = deportista.union(deportista2)
deportista.take(4)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273']]

Despues procedemos a eliminar el encabezado

In [20]:
deportista = deportista.mapPartitionsWithIndex( dropFirstRow ) # mapPartitionsWithIndex permite particionar un rdd
deportista.take(4)

[['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705']]

Ahora tranformamos cada valor a su respectivo tipo de dato.

In [21]:
deportista = deportista.map(lambda l: (
   int(l[0]), l[1], int(l[2]), int(l[3]), int(l[4]), float(l[5]), int(l[6])
))
deportista.take(4)

[(2, 'A Lamusi', 1, 23, 170, 60.0, 199),
 (3, 'Gunnar Nielsen Aaby', 1, 24, 0, 0.0, 273),
 (4, 'Edgar Lindenau Aabye', 1, 34, 0, 0.0, 278),
 (5, 'Christine Jacoba Aaftink', 2, 21, 185, 82.0, 705)]

Creamos el Schema respectivo.

In [22]:
deportista_schema = StructType([
  StructField('deportista_id', IntegerType(), False),
  StructField('nombre', StringType(), False),
  StructField('genero', IntegerType(), False),
  StructField('edad', IntegerType(), True),
  StructField('altura', IntegerType(), True),
  StructField('peso', FloatType(), True),
  StructField('equipo_id', IntegerType(), True)
])

deportista = sql_context.createDataFrame(deportista, deportista_schema)
deportista.show(4)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 4 rows



In [10]:
# spark.stop()
